# CH14 SVD

## Dependencies

In [ ]:
import pandas as pd
import numpy as np
from numpy import linalg as la


SVD (Singular Value Decomposition)奇异值分解
- 隐性语义索引（Latent Semantic Indexing, LSI）或者 隐性语义分析(Latent Semantic Analysis, LSA)
- 推荐系统


- PCA -> 特征值
- SVD -> 奇异值


## MLiA

### SVD Applications

#### LSI


在模式识别和信息检索中，SVD在降维方面有广泛应用，被称为是浅语义索引的基础。

#### RS
特征生成，将数据映射成重要特征。低维的数据饱含了整个数据的大部分能量

### Matrix Decomposition

In [ ]:
U, sigma, VT = np.linalg.svd([[1, 1],[7, 7]])

In [ ]:
U, sigma, VT

### CF

collaborative filtering

#### **Similarity**
- 欧式距离
    - [0, 1]
- Jaccard相似度
- 皮尔逊相关系数
     - 量级不敏感
     - [-1, 1]
- 余弦相似度
    - [-1, 1]

In [ ]:
def ecludSim(inA, inB):
    return 1/(1 + la.norm(inA -inB))

def pearsSim(inA, inB):
    if len(inA) < 3:
        return 1
    return 0.5+0.5*np.corrcoef(inA, inB, rowvar=0)[0][1]

def cosSim(inA, inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [ ]:
X = np.mat([[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]])


In [ ]:
X[:, 0]

In [ ]:
ecludSim(X[:, 0], X[:, 4]), ecludSim(X[:, 0], X[:, 0])

In [ ]:
pearsSim(X[:, 0], X[:, 4]), pearsSim(X[:, 0], X[:, 0])

In [ ]:
cosSim(X[:, 0], X[:, 4]), cosSim(X[:, 0], X[:, 0])

Item based recommondation

#### Item or User

- user-based CF
- item-based CF

如果用户数量很多，往往通过Item Similarity来实现推荐

#### Cost
- 打分 : RMSE
- TopN : Precision 和 Recall
- 覆盖率 : 熵
- 多样性 : 

### Ex : 餐饮推荐

#### Scoring for unscored

##### copy

In [ ]:
X.shape

In [ ]:
mX = X

In [ ]:
mX[0,1] = 4

In [ ]:
X, mX

In [ ]:
mX = X.copy()

In [ ]:
mX[0, 1] = 5
mX, X

In [ ]:
mX[0, 1] = mX[0, 0]= mX[1, 0]=mX[2, 0]=4
mX[3, 3] = 2

##### modified X

In [ ]:
mX

In [ ]:
mX[:,1],mX[:,1].A

In [ ]:
mX[0,1]

##### overLaped users
overlap means which user have scores at item and j both

In [ ]:
item = 1
j = 2
overLap = np.nonzero(np.logical_and(mX[:, item].A>0,
                                    mX[:, j].A>0))[0]
mX[:, item].A,mX[:,j].A, overLap

In [ ]:
def standEst(dataMat, user, simMeas, item):
    n = dataMat.shape[1]
    simTotal = 0
    ratSimTotal = 0
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        # 筛选用户： 在item 和 j两列都有值的用户
        overLap = np.nonzero(np.logical_and(dataMat[:, item].A>0,
                                            dataMat[:, j].A>0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            # 通过都有值的用户数据，计算item和j列的相似度
            similarity = simMeas(dataMat[overLap, item],
                                 dataMat[overLap, j])
            print("the %d and %d similarity is : %f" % (item, j, similarity))
            # item-based，基于item之间的相似度，通过其他item的打分，获取打分
            simTotal += similarity
            # 基于相似度，通过用户给j列的打分，预测给item的打分
            ratSimTotal += similarity*userRating
    return 0 if simTotal == 0 else ratSimTotal/simTotal

##### nonzero

In [ ]:
mX

In [ ]:
user =  1
mX[user,:].A == 0

In [ ]:
mX[user,:]

In [ ]:
np.nonzero(mX[user,:].A == 0)

In [ ]:
user =  4
mX[user,:].A == 0

In [ ]:
np.nonzero(mX[user,:].A == 0)

##### recommend

In [ ]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0: 
        return "you rated everything"
    itemScores = [(item, estMethod(dataMat, user, simMeas, item)) for item in unratedItems]
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]
        

In [ ]:
recommend(mX, 2)

#### SVD

In [ ]:
df = pd.read_csv("./Data/CH14/data_14-4.csv",index_col=0)
df

##### SVD
转化成低维矩阵计算相似度

In [ ]:
X = np.mat(df.values)
X

In [ ]:
U, Sigma, VT = la.svd(X)
Sigma

In [ ]:
sum(Sigma),sum(Sigma)*.9

In [ ]:
sum(Sigma[:2]),sum(Sigma[:3]),sum(Sigma[:4]),sum(Sigma[:5]),sum(Sigma[:6]),sum(Sigma[:7])

##### Sigma

In [ ]:
Sig4 = np.mat(np.eye(4)*Sigma[:4])
Sig4

In [ ]:
Sig5 = np.mat(np.eye(5)*Sigma[:5])
Sig5

In [ ]:
Sig5.I

##### Transformed

In [ ]:
xformedItems = X.T*U[:,:5]*Sig5.I
xformedItems

In [ ]:
def svdEst(dataMat, user, simMeas, item):
    n = dataMat.shape[1]
    simTotal = 0
    ratSimTotal = 0
    U, Sigma, VT = la.svd(dataMat)
    n_Sig = 6
    Sig = np.mat(np.eye(n_Sig)*Sigma[:n_Sig])
    xformedItems = dataMat.T*U[:,:n_Sig]*Sig.I
    print(dataMat[user])
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0 or j==item:
            continue
        # 通过都有值的用户数据，计算item和j列的相似度
        similarity = simMeas(xformedItems[item,:].T,
                             xformedItems[j,:].T)
        print("the %d and %d similarity is : %f" % (item, j, similarity))
        # item-based，基于item之间的相似度，通过其他item的打分，获取打分
        simTotal += similarity
        # 基于相似度，通过用户给j列的打分，预测给item的打分
        ratSimTotal += similarity*userRating
    return 0 if simTotal == 0 else ratSimTotal/simTotal

##### recommend

In [ ]:
recommend(X,0,estMethod=svdEst)

In [ ]:
recommend(X,1,estMethod=svdEst)

#### Summary

- 大数据集SVD操作低频运行
- Sparse
- Similarity 存储
- Cold-Start 推荐转化成搜索

### Ex : 图像压缩

## Sklearn

In [ ]:
from skimage import data, io, filters
image = data.coins()
# ... or any other NumPy array!
edges = filters.sobel(image)
io.imshow(edges)
io.show()